## Create a simple tensor with random items

In [5]:
import numpy as np

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Generate randomly distributed parameters
params = np.random.uniform(low=-10, high=150, size=20)

# Make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

# Print the parameters
print(params)

[150.33  -8.79   0.   149.33  -7.79  -0.47 143.6  148.05  78.39 119.07
  88.94 111.48  88.98  -3.51  74.04 115.75  58.34  18.91 119.3  102.99]


## Define the quantization methods and quantize

In [6]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)

(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[150.33  -8.79   0.   149.33  -7.79  -0.47 143.6  148.05  78.39 119.07
  88.94 111.48  88.98  -3.51  74.04 115.75  58.34  18.91 119.3  102.99]

Asymmetric scale: 0.624, zero: 14.0
[255   0  14 253   2  13 244 251 140 205 157 193 157   8 133 199 107  44
 205 179]

Symmetric scale: 1.1837007874015748
[127  -7   0 126  -7   0 121 125  66 101  75  94  75  -3  63  98  49  16
 101  87]


In [7]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[150.33  -8.79   0.   149.33  -7.79  -0.47 143.6  148.05  78.39 119.07
  88.94 111.48  88.98  -3.51  74.04 115.75  58.34  18.91 119.3  102.99]

Dequantize Asymmetric:
[150.38  -8.74   0.   149.14  -7.49  -0.62 143.52 147.89  78.62 119.18
  89.23 111.7   89.23  -3.74  74.26 115.44  58.03  18.72 119.18 102.96]

Dequantize Symmetric:
[150.33  -8.29   0.   149.15  -8.29   0.   143.23 147.96  78.12 119.55
  88.78 111.27  88.78  -3.55  74.57 116.    58.    18.94 119.55 102.98]


In [8]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.04
   Symmetric error: 0.09
